In [152]:
import pandas as pd
import json
import joblib

# Load Dataset

In [153]:
with open('/content/Intent.json') as f:
    data = json.load(f)

In [154]:
df = pd.DataFrame(data['intents'])

In [155]:
df.head()

,intent,text,responses,extension,context,entityType,entities
0,Greeting,"[Hi, Hi there, Hola, Hello, Hello there, Hya, ...","[Hi human, please tell me your GeniSys user, H...","{'function': '', 'entities': False, 'responses...","{'in': '', 'out': 'GreetingUserRequest', 'clea...",NA,[]
1,GreetingResponse,"[My user is Adam, This is Adam, I am Adam, It ...","[Great! Hi <HUMAN>! How can I help?, Good! Hi ...","{'function': 'extensions.gHumans.updateHuman',...","{'in': 'GreetingUserRequest', 'out': '', 'clea...",NA,"[{'entity': 'HUMAN', 'rangeFrom': 3, 'rangeTo'..."
2,CourtesyGreeting,"[How are you?, Hi how are you?, Hello how are ...","[Hello, I am great, how are you? Please tell m...","{'function': '', 'entities': False, 'responses...","{'in': '', 'out': 'CourtesyGreetingUserRequest...",NA,[]
3,CourtesyGreetingResponse,"[Good thanks! My user is Adam, Good thanks! Th...","[Great! Hi <HUMAN>! How can I help?, Good! Hi ...","{'function': 'extensions.gHumans.updateHuman',...","{'in': 'GreetingUserRequest', 'out': '', 'clea...",NA,"[{'entity': 'HUMAN', 'rangeFrom': 5, 'rangeTo'..."
4,CurrentHumanQuery,"[What is my name?, What do you call me?, Who d...","[You are <HUMAN>! How can I help?, Your name i...",{'function': 'extensions.gHumans.getCurrentHum...,"{'in': '', 'out': 'CurrentHumanQuery', 'clear'...",NA,[]


In [156]:
df = df[["intent", "text"]]
df.head()

,intent,text
0,Greeting,"[Hi, Hi there, Hola, Hello, Hello there, Hya, ..."
1,GreetingResponse,"[My user is Adam, This is Adam, I am Adam, It ..."
2,CourtesyGreeting,"[How are you?, Hi how are you?, Hello how are ..."
3,CourtesyGreetingResponse,"[Good thanks! My user is Adam, Good thanks! Th..."
4,CurrentHumanQuery,"[What is my name?, What do you call me?, Who d..."


In [157]:
df = df.explode('text')
df.head()

,intent,text
0,Greeting,Hi
0,Greeting,Hi there
0,Greeting,Hola
0,Greeting,Hello
0,Greeting,Hello there


In [158]:
df.reset_index(drop=True, inplace=True)
df.head()

,intent,text
0,Greeting,Hi
1,Greeting,Hi there
2,Greeting,Hola
3,Greeting,Hello
4,Greeting,Hello there


# Preprocess Text

In [159]:
import nltk
import string
import re

from nltk.stem.porter import PorterStemmer

nltk.download('stopwords')

from nltk.corpus import stopwords

nltk.download('punkt')

ps = PorterStemmer()

def clean_text(text):
    text = text.lower()
    text = re.sub(r'http.?://[^\s]+[\s]?', '', text)
    text = nltk.word_tokenize(text)
    y = []

    for i in text:
      if i not in stopwords.words('english'):
        y.append(i)

    text = y[:]
    y.clear()

    for i in text:
        y.append(ps.stem(i))

    return " ".join(y)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [160]:
df['text_cleaned'] = df['text'].apply(clean_text)
df.head()

,intent,text,text_cleaned
0,Greeting,Hi,hi
1,Greeting,Hi there,hi
2,Greeting,Hola,hola
3,Greeting,Hello,hello
4,Greeting,Hello there,hello


In [161]:
df.groupby('intent').describe()

text                                             \
                         count unique                                 top   
intent                                                                      
Clever                       7      7                 You are very clever   
CourtesyGoodBye              6      6                         Thanks, bye   
CourtesyGreeting             7      7                        How are you?   
CourtesyGreetingResponse     8      8        Good thanks! My user is Adam   
CurrentHumanQuery            7      7                    What is my name?   
GoodBye                      4      4                                 Bye   
Gossip                       6      6           I am bored gossip with me   
Greeting                     7      7                                  Hi   
GreetingResponse             8      8                     My user is Adam   
Jokes                        6      6                      Tell me a joke   
NameQuery                    6      6                  What is your name?   
NotTalking2U                 7      7             I am not talking to you   
PodBayDoor                   7      7               Open the pod bay door   
PodBayDoorResponse           7      7                                 Why   
RealNameQuery                7      6              Your real name please?   
SelfAware                    7      7    Can you prove you are self-aware   
Shutup                       7      7                            Be quiet   
Swearing                     4      4                            fuck off   
Thanks                       6      6                        OK thank you   
TimeQuery                    7      7                   What is the time?   
UnderstandQuery              6      6  Do you understand what I am saying   
WhoAmI                       6      6                     Can you see me?   

                              text_cleaned                                 
                         freq        count unique                top freq  
intent                                                                     
Clever                      1            7      5        clever girl    2  
CourtesyGoodBye             1            6      5        thank , bye    2  
CourtesyGreeting            1            7      6                  ?    2  
CourtesyGreetingResponse    1            8      4  good thank ! adam    3  
CurrentHumanQuery           1            7      6            think ?    2  
GoodBye                     1            4      4                bye    1  
Gossip                      1            6      5        tell gossip    2  
Greeting                    1            7      4                 hi    2  
GreetingResponse            1            8      4               adam    3  
Jokes                       1            6      6          tell joke    1  
NameQuery                   1            6      6             name ?    1  
NotTalking2U                1            7      5               talk    3  
PodBayDoor                  1            7      3  open pod bay door    3  
PodBayDoorResponse          1            7      5                       2  
RealNameQuery               2            7      4  real name pleas ?    3  
SelfAware                   1            7      6     prove consciou    2  
Shutup                      1            7      6              quiet    2  
Swearing                    1            4      3               fuck    2  
Thanks                      1            6      4           ok thank    2  
TimeQuery                   1            7      5        know time ?    2  
UnderstandQuery             1            6      6     understand say    1  
WhoAmI                      1            6      4              see ?    2

# Feature Extraction

In [162]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [163]:
tf_vec = TfidfVectorizer(max_features=3000)

In [164]:
X = tf_vec.fit_transform(df['text_cleaned']).toarray()

In [165]:
Y = df['intent'].values

In [166]:
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

# Train Model

In [167]:
from sklearn.model_selection import train_test_split

In [168]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=2)

In [169]:
from sklearn.metrics import accuracy_score

In [170]:
from sklearn.ensemble import RandomForestClassifier

In [171]:
model = RandomForestClassifier()
model.fit(X_train,y_train)

RandomForestClassifier()

In [172]:
y_pred = model.predict(X_test)
print(accuracy_score(y_test,y_pred))

0.5277777777777778


In [173]:
def get_input(text):
  text = clean_text(text)
  text = pd.Series(text)
  x_pred = tf_vec.transform(text)
  return x_pred

In [174]:
class model_intent():
  def predict_intent(text):
    pred = get_input(text)
    y_pr = model.predict_proba(pred).max()
    if (y_pr < 0.4):
      print("NLU fallback: Intent could not be confidently determined")
    else:
      print(model.predict(pred))

In [175]:
filename = 'model.sav'
joblib.dump(model_intent, filename)

['model.sav']

In [176]:
loaded_model = joblib.load(filename)
result = loaded_model.predict_intent("Who are you?")

['CourtesyGreeting']
